In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc

%matplotlib inline

In [2]:
# create long representation for file and cust
#march = pd.read_hdf('./march.h5', 'df')
#march = march.groupby(['file_id','customer_id']).count().reset_index()[['file_id','customer_id','timestamp']]

#april = pd.read_hdf('./april.h5', 'df')
#april = april.groupby(['file_id','customer_id']).count().reset_index()[['file_id','customer_id','timestamp']]

#may = pd.read_hdf('./may.h5', 'df')
#may = may.groupby(['file_id','customer_id']).count().reset_index()[['file_id','customer_id','timestamp']]

#data = pd.concat([march,april,may],axis=0)

#data = data.groupby(['file_id','customer_id']).count().reset_index()[['file_id','customer_id']]
#data.head(3)

,file_id,customer_id
0,00008c73ee43c15b16c26b26398c1577,033e91e730e176cd2ac81dc48a8f5088
1,00008c73ee43c15b16c26b26398c1577,06bea9f9c1d72c4d3151efa6bcd5ba83
2,00008c73ee43c15b16c26b26398c1577,113597f904c023f44d1a01babb780f58


In [3]:
data.to_hdf('file_cust_longrep.h5','df')
data = pd.read_hdf('file_cust_longrep.h5', 'df')
data.head(3)

,file_id,customer_id
0,00008c73ee43c15b16c26b26398c1577,033e91e730e176cd2ac81dc48a8f5088
1,00008c73ee43c15b16c26b26398c1577,06bea9f9c1d72c4d3151efa6bcd5ba83
2,00008c73ee43c15b16c26b26398c1577,113597f904c023f44d1a01babb780f58


In [4]:
file_df = pd.read_csv('file_df.csv')
file_df = file_df[['file_id','first_day']]

data = pd.merge(data,file_df,on='file_id',how='left')

train = pd.read_csv('training-set.csv',header=None)
test = pd.read_csv('testing-set.csv',header=None)
train.columns = ['file_id','target']
test.columns = ['file_id','target']

### full mean code
#fullmeancode = pd.read_csv('final/forfullmeancode.csv')
#train = pd.concat([train,fullmeancode],axis=0)
### full mean code

data = pd.merge(data,train,how='left',on='file_id')
data.head(3)

,file_id,customer_id,first_day,target
0,00008c73ee43c15b16c26b26398c1577,033e91e730e176cd2ac81dc48a8f5088,76,NaN
1,00008c73ee43c15b16c26b26398c1577,06bea9f9c1d72c4d3151efa6bcd5ba83,76,NaN
2,00008c73ee43c15b16c26b26398c1577,113597f904c023f44d1a01babb780f58,76,NaN


In [5]:
data = data.sort_values(by=['first_day','file_id']).reset_index(drop=True)
data.head(3)

,file_id,customer_id,first_day,target
0,0087304d5bee451f0d82c7407b0fa7d1,282396145a3df4452761bacf8049f6db,1,0.0
1,0087304d5bee451f0d82c7407b0fa7d1,501782bb051c5368f641f5299e550f32,1,0.0
2,0087304d5bee451f0d82c7407b0fa7d1,54a57555a85251b45a9de37ba95a4cf3,1,0.0


In [6]:
tr_data = data[data['first_day']<50] # 62 for predict, 50 for val
tr_data = tr_data[['file_id','customer_id','target']]
tr_data.head(3)

,file_id,customer_id,target
0,0087304d5bee451f0d82c7407b0fa7d1,282396145a3df4452761bacf8049f6db,0.0
1,0087304d5bee451f0d82c7407b0fa7d1,501782bb051c5368f641f5299e550f32,0.0
2,0087304d5bee451f0d82c7407b0fa7d1,54a57555a85251b45a9de37ba95a4cf3,0.0


In [7]:
cumsum = tr_data.groupby('customer_id').cumsum().target - tr_data['target']
cumcnt = tr_data.groupby('customer_id').cumcount()
tr_data['mean_score'] = cumsum / cumcnt

In [8]:
tr_data['mean_score'].mean()

0.005896548369626853

In [9]:
alpha = 20
gbmean = 0.006764493003441334

tr_data['mean_score'] = ((tr_data['mean_score']*cumcnt)+(gbmean*alpha))/(cumcnt+alpha)

In [10]:
customer_score = tr_data.groupby('customer_id').last().reset_index()[['customer_id','mean_score']]
customer_score.head(3)

,customer_id,mean_score
0,00000145d9062eada528bace5fb4864e,NaN
1,000003d77bee692d83b7b83fdba60d3d,0.003146
2,00000aea7887eaf96dbd022025e426fd,NaN


In [11]:
customer_df = pd.read_csv('customer_df.csv')
customer_df = pd.merge(customer_df,customer_score,how='left',on='customer_id')
customer_df = customer_df[['customer_id','mean_score']]
customer_df.head(3)

,customer_id,mean_score
0,0000006fa286976bf35ea17f1f19bc7a,NaN
1,00000145d9062eada528bace5fb4864e,NaN
2,000003d77bee692d83b7b83fdba60d3d,0.003146


In [12]:
#customer_df.loc[pd.isnull(customer_df).mean_score,'mean_score']=0.012656403967587571
customer_df = customer_df[customer_df['mean_score']>-1]
customer_df.shape

(2529080, 2)

In [13]:
te_data = data[data['first_day']>49]
te_data.head(3)

,file_id,customer_id,first_day,target
35808811,0021c7f072ad0e8068ab8c611700bfe1,00546d10281534af9f1bcbb14dca53d6,50,0.0
35808812,0021c7f072ad0e8068ab8c611700bfe1,04a890cd36f3cface3faeea2fc06967b,50,0.0
35808813,0021c7f072ad0e8068ab8c611700bfe1,052c7bc0c271b8f1accb80ca2bda46db,50,0.0


In [14]:
te_data = te_data.sort_values(by=['first_day','file_id']).reset_index(drop=True)
te_data = te_data[['file_id','customer_id']]
te_data = pd.merge(te_data,customer_df,how='left',on='customer_id')
te_data.head(3)

,file_id,customer_id,mean_score
0,0021c7f072ad0e8068ab8c611700bfe1,00546d10281534af9f1bcbb14dca53d6,NaN
1,0021c7f072ad0e8068ab8c611700bfe1,04a890cd36f3cface3faeea2fc06967b,0.004665
2,0021c7f072ad0e8068ab8c611700bfe1,052c7bc0c271b8f1accb80ca2bda46db,0.005203


In [15]:
data = pd.concat([tr_data,te_data],ignore_index=True)
data.head(3)

,customer_id,file_id,mean_score,target
0,282396145a3df4452761bacf8049f6db,0087304d5bee451f0d82c7407b0fa7d1,NaN,0.0
1,501782bb051c5368f641f5299e550f32,0087304d5bee451f0d82c7407b0fa7d1,NaN,0.0
2,54a57555a85251b45a9de37ba95a4cf3,0087304d5bee451f0d82c7407b0fa7d1,NaN,0.0


In [16]:
#data.loc[pd.isnull(data).mean_score,'mean_score']=0.012656403967587571
data = data[data['mean_score']>-1]
data.head(3)

,customer_id,file_id,mean_score,target
455,282396145a3df4452761bacf8049f6db,00a0fadf6f4b4d244459b3bc8f6ee7fa,0.006442,0.0
467,282396145a3df4452761bacf8049f6db,00a885e87beb6fb1f6ca6ee9eb783779,0.006150,0.0
528,282396145a3df4452761bacf8049f6db,00a92b584bfcf58b70d5e965e91da03c,0.005882,0.0


In [17]:
data = data[['file_id','mean_score']]
exp_menc_cust = data.groupby('file_id').mean().reset_index()
exp_menc_cust.head(3)

,file_id,mean_score
0,00008c73ee43c15b16c26b26398c1577,0.010983
1,0000e2398b12121a85166fed5fe2a3da,0.006150
2,0001fe8dce14ce099aa6ca8ea5026ea7,0.005141


In [18]:
train = pd.read_csv('training-set.csv',header=None)
test = pd.read_csv('testing-set.csv',header=None)
train.columns = ['file_id','target']
test.columns = ['file_id','target']
trte = pd.concat([train,test],axis=0)
trte = pd.merge(trte,exp_menc_cust,how='left',on='file_id')
trte = trte[['file_id','mean_score']]
trte.head(3)

,file_id,mean_score
0,0000e2398b12121a85166fed5fe2a3da,0.006150
1,0001fe8dce14ce099aa6ca8ea5026ea7,0.005141
2,00027f50019000accc492e5684efc818,0.019314


In [19]:
trte.columns = ['file_id','cust_mean']

In [20]:
trte['cust_mean'].mean()

0.03514704668048272

In [21]:
trte.loc[pd.isnull(trte)['cust_mean'],'cust_mean']=0.03535477004433718

In [22]:
trte.to_csv('cust_expsmo_meancode_val.csv',index=False)
trte.shape

(81894, 2)